# Проект «Анализ вакансий из HeadHunter»
   

In [126]:
pip install psycopg2

In [127]:
import pandas as pd
import psycopg2

In [128]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [129]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [130]:
cur = connection.cursor()

##  Анализ данных

1. Сколько вакансий есть в базе?

In [131]:

query_3_1 = f'''
SELECT count(*) FROM public.vacancies
'''
cur.execute(query_3_1)
cur.fetchone()

(49197,)

2.Сколько работодателей?

In [132]:

query_3_2 = f'''
SELECT count(*) FROM public.employers
'''
cur.execute(query_3_2)
cur.fetchone()

(23501,)

3. Сколько регионов?

In [133]:

query_3_3 = f'''
SELECT count(*) FROM public.areas
'''
cur.execute(query_3_3)
cur.fetchone()

(1362,)

4. Сколько сфер деятельности?

In [134]:

query_3_4 = f'''
SELECT count(*) FROM public.industries
'''
cur.execute(query_3_4)
cur.fetchone()

(294,)

***

Проведён первичный обзор данных, который позволил оценить масштаб и структуру анализируемой базы вакансий.

Объём данных: База содержит сведения о 49 197 вакансиях, что формирует солидную основу для статистического анализа.

Работодатели: Вакансии опубликованы от имени 3 501 работодателя. Это указывает на значительное разнообразие компаний-нанимателей в выборке.

География: Данные охватывают 1 362 региона, что свидетельствует о широком географическом распределении вакансий и возможности анализа региональных рынков труда.

Отрасли: Вакансии относятся к 294 различным сферам деятельности (индустриям), отражая широкий профессиональный спектр.

Выводы по предварительному анализу:

Репрезентативность выборки: Объём данных (почти 50 тысяч вакансий) и их охват (тысячи работодателей, регионов и сфер) позволяют считать базу достаточно представительной для проведения содержательного анализа рынка труда.

Потенциал для детализации: Наличие детализации по отраслям и регионам открывает возможности для глубокой сегментации: можно сравнивать спрос в разных индустриях, выявлять регионы-лидеры по количеству вакансий или анализировать активность конкретных работодателей.

Соотношение вакансий и компаний: В среднем на одного работодателя приходится около 14 вакансий. Это может указывать как на высокую активность крупных компаний в подборе персонала, так и на наличие множества малых предприятий с единичными вакансиями.

Направление дальнейшего анализа: Первичный обзор задаёт ключевые векторы для углублённой аналитики: изучение наиболее популярных сфер деятельности, определение регионов с наибольшей концентрацией вакансий, анализ распределения вакансий по работодателям (например, выявление наиболее активных рекрутеров) и исследование отраслевой структуры спроса.

## 4. Анализ вакансий

In [135]:
#сколько (cnt) вакансий в каждом регионе (area). Отсортируйте по количеству вакансий в порядке убывания.
query_4_1 = f'''
SELECT
    a.name AS area,
    count(v.id) AS cnt
FROM public.areas a
LEFT JOIN public.vacancies v ON a.id = v.area_id
GROUP BY a.id, a.name
ORDER BY cnt DESC
'''

In [136]:

cur.execute(query_4_1)
results = cur.fetchmany(5)
for row in results:
    print(row)

('Москва', 5333)
('Санкт-Петербург', 2851)
('Минск', 2112)
('Новосибирск', 2006)
('Алматы', 1892)


2. Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [137]:

query_4_2 = f'''
SELECT count(*)
FROM public.vacancies
WHERE salary_from IS NOT NULL
   OR salary_to IS NOT NULL
'''
cur.execute(query_4_2)
cur.fetchone()

(24073,)

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [138]:
# текст запроса
query_4_3 = f'''
SELECT
    ROUND(AVG(salary_from)) AS avg_salary_from,
    ROUND(AVG(salary_to)) AS avg_salary_to
FROM public.vacancies
WHERE salary_from IS NOT NULL
   OR salary_to IS NOT NULL
'''
cur.execute(query_4_3)
cur.fetchone()

(Decimal('71065'), Decimal('110537'))

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [139]:

query_4_4 = f'''
SELECT
    schedule,
    employment,
    count(*) AS vacancy_count
FROM public.vacancies
WHERE schedule IS NOT NULL
    AND employment IS NOT NULL
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
'''
cur.execute(query_4_4)
results = cur.fetchmany(2)
for row in results:
    print(row)

('Полный день', 'Полная занятость', 35367)
('Удаленная работа', 'Полная занятость', 7802)


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [140]:

query_4_5 = f'''
SELECT
    experience,
    count(*) AS vacancy_count
FROM public.vacancies
WHERE experience IS NOT NULL
GROUP BY experience
ORDER BY vacancy_count ASC
'''
cur.execute(query_4_5)
results = cur.fetchmany(4)
for row in results:
    print(row)

('Более 6 лет', 1337)
('Нет опыта', 7197)
('От 3 до 6 лет', 14511)
('От 1 года до 3 лет', 26152)


# Выводы по детальному анализу структуры вакансий
Детальный анализ ключевых параметров вакансий позволяет сформировать следующие выводы о структуре и тенденциях рынка труда в представленной выборке:

## 1. Концентрация вакансий в крупнейших центрах и международный охват.

На рынке доминируют столичные мегаполисы: Москва (5 333 вакансии) является абсолютным лидером, формируя почти вдвое больше предложений, чем Санкт-Петербург (2 851).

В Топ-5 также входят крупнейшие города других стран СНГ — Минск (Беларусь) и Алматы (Казахстан), что подчеркивает транснациональный характер анализируемой базы данных и указывает на схожие тренды на рынках труда этих стран.

##2. Низкая прозрачность в вопросах оплаты труда.

Критически низкий показатель: только 48.9% вакансий (24 073 из 49 197) содержат информацию о зарплате. Это свидетельствует о системной проблеме отсутствия зарплатной прозрачности, что усложняет процесс поиска работы для соискателей и искажает реальную картину рыночных доходов.

##3. Значительный разброс в зарплатных предложениях.

Средняя зарплатная вилка имеет широкий диапазон: от 71 065 руб. (нижняя граница) до 110 537 руб. (верхняя). Разрыв в ~39,5 тыс. рублей указывает на то, что итоговый оклад часто является предметом переговоров и сильно зависит от конкретных навыков, опыта и результатов собеседования кандидата.

##4. Доминирование традиционного формата занятости при устоявшемся тренде на удаленную работу.

Классический формат «Полный день» (71.9% или 35 367 вакансий) остается стандартом для большинства позиций.

При этом удаленная работа (15.9% или 7 802 вакансии) закрепилась как значимый и устойчивый сегмент рынка, что подтверждает переход от вынужденной меры к стабильной гибридной модели в целом ряде отраслей.

##5. Рынок ориентирован на молодых специалистов с минимальным опытом.

Наибольший спрос предъявляется к специалистам с опытом от 1 до 3 лет (26 152 вакансии, более 50% от выборки с указанным опытом). Это говорит о том, что компании активно растут и ищут исполнителей, уже прошедших начальную адаптацию, но не претендующих на уровень старших экспертов.

Сегмент «Нет опыта» (7 197 вакансий) также является существенным, что указывает на наличие возможностей для старта карьеры и готовность части работодателей инвестировать в обучение новичков.

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [141]:

query_5_1 = f'''
SELECT
    e.name AS employer_name,
    count(v.id) AS vacancy_count
FROM public.employers e
LEFT JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
ORDER BY vacancy_count DESC
LIMIT 5
'''
cur.execute(query_5_1)
results = cur.fetchmany(5)
for row in results:
    print(row)

('Яндекс', 1933)
('Ростелеком', 491)
('Тинькофф', 444)
('СБЕР', 428)
('Газпром нефть', 331)


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [142]:

query_5_2 = f'''
SELECT
    a.id AS area_id,
    a.name AS region,
    count(distinct e.id) AS employer_count,
    count(v.id) AS vacancy_count
FROM public.areas a
LEFT JOIN public.vacancies v ON a.id = v.area_id
LEFT JOIN public.employers e ON a.id = e.area
GROUP BY a.id, a.name
HAVING count(v.id) = 0
ORDER BY count(distinct e.id) DESC
LIMIT 1
'''
cur.execute(query_5_2)
cur.fetchone()

(113, 'Россия', 410, 0)

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [143]:

query_5_3 = f'''
SELECT
    e.name AS employer_name,
    count(distinct v.area_id) AS regions_count
FROM public.employers e
LEFT JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
HAVING count(distinct v.area_id) > 0
ORDER BY regions_count DESC
'''
cur.execute(query_5_3)
cur.fetchone()

('Яндекс', 181)

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [144]:

query_5_4 = f'''
SELECT count(*) AS employers_without_industry
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''
cur.execute(query_5_4)
cur.fetchone()

(8419,)

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [145]:

query_5_5 = f'''
WITH employers_industry_count AS (
    SELECT
        e.id,
        e.name,
        count(ei.industry_id) AS industry_count
    FROM public.employers e
    INNER JOIN public.employers_industries ei ON e.id = ei.employer_id
    GROUP BY e.id, e.name
    HAVING count(ei.industry_id) = 4
)
SELECT name
FROM employers_industry_count
ORDER BY name
OFFSET 2
LIMIT 3
'''
cur.execute(query_5_5)
results = cur.fetchmany(3)
for row in results:
    print(row)

('2ГИС',)
('2К',)
('4 пикселя +',)


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [146]:

query_5_6 = f'''
SELECT count(distinct ei.employer_id) AS employers_count
FROM public.employers_industries ei
INNER JOIN public.industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''
cur.execute(query_5_6)
cur.fetchone()

(3553,)

7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [147]:

query_5_7 = f'''
WITH million_cities AS (
    SELECT unnest(array[
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
        'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск',
        'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону',
        'Омск', 'Воронеж', 'Пермь', 'Волгоград'
    ]) AS city_name
),
yandex_vacancies_by_city AS (
    SELECT
        a.name AS city,
        count(v.id) AS cnt
    FROM public.vacancies v
    INNER JOIN public.employers e ON v.employer_id = e.id
    INNER JOIN public.areas a ON v.area_id = a.id
    WHERE e.name like '%Яндекс%'
      AND a.name IN (SELECT city_name FROM million_cities)
    GROUP BY a.name
),
combined_results AS (
    SELECT
        city AS area,
        cnt,
        0 as sort_order
    FROM yandex_vacancies_by_city

    UNION ALL

    SELECT
        'Total' AS area,
        SUM(cnt) AS cnt,
        1 as sort_order
    FROM yandex_vacancies_by_city
)
SELECT
    area,
    cnt
FROM combined_results
ORDER BY
    sort_order,
    cnt ASC;
'''
cur.execute(query_5_7)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Омск', Decimal('21'))
('Челябинск', Decimal('22'))
('Красноярск', Decimal('23'))
('Волгоград', Decimal('24'))
('Пермь', Decimal('25'))
('Казань', Decimal('25'))
('Ростов-на-Дону', Decimal('25'))
('Самара', Decimal('26'))
('Уфа', Decimal('26'))
('Краснодар', Decimal('30'))
('Воронеж', Decimal('32'))
('Новосибирск', Decimal('35'))
('Нижний Новгород', Decimal('36'))
('Екатеринбург', Decimal('39'))
('Санкт-Петербург', Decimal('43'))
('Москва', Decimal('59'))
('Total', Decimal('491'))


***

# Выводы по анализу работодателей
## 5. Анализ работодателей

Анализ данных о работодателях выявляет структуру спроса на рынке труда, стратегии крупнейших игроков и ключевые проблемы в качестве данных.

##1. Явный лидер рынка и поляризация спроса.

«Яндекс» является безусловным лидером, формируя 1 933 вакансии, что в 4 раза превышает активность ближайшего конкурента («Ростелеком», 491 вакансия). Вместе с «Тинькофф» и «СБЕР» он формирует топ IT-сектора, в то время как «Газпром нефть» представляет традиционную корпорацию. Это указывает на значительное влияние крупного технологического бизнеса на общий объем вакансий в базе.

##2. Уникальное географическое покрытие лидера.

«Яндекс» демонстрирует беспрецедентную для выборки географическую экспансию, публикуя вакансии в 181 регионе (13.3% от всех). Его присутствие во всех 16 городах-миллионниках подтверждает стратегию создания распределенных команд (хабов) по всей стране, а не концентрации исключительно в столицах.

##3. Стратегическая дисперсия vs. столичная концентрация.

Распределение вакансий «Яндекса» по миллионникам опровергает стереотип о гиперконцентрации в Москве: на столицу приходится лишь 59 из 491 вакансии (12%) в этих городах. Относительно равномерное распределение (от 21 вакансии в Омске до 59 в Москве) свидетельствует о сознательной децентрализации талантов и развитии ИТ-кластеров в регионах.

##4. Критическая проблема с указанием сфер деятельности.

Тревожный показатель: 8 419 работодателей (35.8%) не указали свою сферу деятельности. Это существенно снижает точность отраслевого анализа и затрудняет для соискателей и аналитиков сегментацию рынка. Требуются меры по верификации и дополнению данных.

##5. Доминирование IT-сектора в заявленной специализации.

Сфера «Разработка программного обеспечения» является самой часто указываемой (3 553 работодателя), что служит еще одним статистическим подтверждением высокого спроса и предложения в IT-отрасли в рамках данной базы данных.

##6. Феномен «спящих» работодателей.

Наличие 410 работодателей из региона «Россия» (код 113) без единой вакансии требует отдельного исследования. Возможные причины: неактуальные записи в базе, компании-«пустышки», работодатели, размещающие вакансии только через другие каналы, или техническая погрешность при сборе данных.

###Общий аналитический вывод:
Анализ подтверждает, что рынок вакансий в значительной степени движется крупными технологическими корпорациями, в частности «Яндексом», который задает тренды как по объему спроса, так и по модели географического распределения. Однако низкое качество заполнения профилей работодателями (отсутствие сферы деятельности) является серьезным ограничением для анализа.

## 6. Предметный анализ

1. Переходим к самому интересному — нам предстоит исследовать требования работодателей к дата-сайентистам.

Сколько вакансий имеет отношение к данным?

In [148]:
query_6_1 = '''
SELECT count(*)
FROM public.vacancies
WHERE LOWER(name) like '%data%'
   OR LOWER(name) like '%данн%'
'''
cur.execute(query_6_1)
result = cur.fetchone()
print(f"Количество вакансий, связанных с данными: {result[0]}")

Количество вакансий, связанных с данными: 1771


2. Определим количество подходящих вакансий для начинающего специалиста в области Data Science.Вакансия относится к области Data Science, если её название содержит хотя бы одну из следующих ключевых фраз:

*   data scientist
*   data science
*   исследователь данных
*   ML (исключая контекст, связанный с HTML)
*   machine learning
*   машинн%обучен%

Критерии отбора вакансий для начинающих (Junior-уровень):
Вакансия считается подходящей для специалиста уровня Junior, если соответствует хотя бы одному из условий:

В названии содержится слово junior.

В требованиях указан опыт работы: «Нет опыта».

Тип трудоустройства указан как «Стажировка».

В рамках последующего анализа мы будем рассматривать только вакансии, удовлетворяющие указанным выше условиям.



In [152]:
query_6_2 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        LOWER(name) like '%data scientist%' OR
        LOWER(name) like '%data science%' OR
        LOWER(name) like '%исследователь данных%' OR
        (LOWER(name) like '%ml%' AND LOWER(name) NOT like '%html%') OR
        LOWER(name) like '%machine learning%' OR
        LOWER(name) like '%машинн%обучен%'
    )
)
SELECT count(*) AS junior_ds_vacancies_count
FROM data_science_vacancies
WHERE (
    -- Условия для Junior уровня
    LOWER(name) like '%junior%' OR
    experience = 'Нет опыта' OR
    employment = 'Стажировка'
)
'''
cur.execute(query_6_2)
cur.fetchone()

(51,)

3. Определить количество вакансий в области Data Science (DS), где в требованиях к кандидату указано владение технологиями SQL или PostgreSQL

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [157]:

query_6_3 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        LOWER(name) LIKE '%data scientist%' OR
        LOWER(name) LIKE '%data science%' OR
        LOWER(name) LIKE '%исследователь данных%' OR
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
        LOWER(name) LIKE '%machine learning%' OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
)
SELECT COUNT(*) AS ds_vacancies_with_sql_count
FROM data_science_vacancies
WHERE (
    LOWER(key_skills) LIKE '%sql%' OR
    LOWER(key_skills) LIKE '%postgres%'
)
'''
cur.execute(query_6_3)
cur.fetchone()

(229,)

4. Определить востребованность языка программирования Python в требованиях к кандидатам на позиции в области Data Science (DS).

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [158]:
# текст запроса
query_6_4 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        LOWER(name) LIKE '%data scientist%' OR
        LOWER(name) LIKE '%data science%' OR
        LOWER(name) LIKE '%исследователь данных%' OR
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
        LOWER(name) LIKE '%machine learning%' OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
)
SELECT COUNT(*) AS ds_vacancies_with_python_count
FROM data_science_vacancies
WHERE (
    key_skills ILIKE '%python%'
)
'''
cur.execute(query_6_4)
cur.fetchone()

(357,)

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [159]:

query_6_5 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        LOWER(name) LIKE '%data scientist%' OR
        LOWER(name) LIKE '%data science%' OR
        LOWER(name) LIKE '%исследователь данных%' OR
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
        LOWER(name) LIKE '%machine learning%' OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
),
skills_count AS (
    SELECT
        CASE
            WHEN key_skills IS NULL OR TRIM(key_skills) = '' THEN 0
            -- Используем регулярное выражение для подсчета непустых элементов
            ELSE array_length(
                regexp_split_to_array(
                    TRIM(key_skills),
                    '\s*,\s*'
                ),
                1
            )
        END AS num_skills
    FROM data_science_vacancies
)
SELECT
    ROUND(AVG(num_skills)::numeric, 2) AS avg_skills_per_ds_vacancy
FROM skills_count;
'''
cur.execute(query_6_5)
cur.fetchone()

<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1225392093.py:53: SyntaxWarning: invalid escape sequence '\s'


(Decimal('0.98'),)

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [160]:

query_6_6 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        LOWER(name) LIKE '%data scientist%' OR
        LOWER(name) LIKE '%data science%' OR
        LOWER(name) LIKE '%исследователь данных%' OR
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
        LOWER(name) LIKE '%machine learning%' OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
),
calculated_salaries AS (
    SELECT
        experience,
        CASE
            WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL
                THEN (salary_from + salary_to) / 2.0
            ELSE COALESCE(salary_from, salary_to)
        END AS calculated_salary
    FROM data_science_vacancies
)
SELECT
    experience,
    ROUND(AVG(calculated_salary)) AS avg_salary
FROM calculated_salaries
GROUP BY experience
ORDER BY experience;
'''
cur.execute(query_6_6)
results = cur.fetchmany(10)
for row in results:
    print(row)

('Более 6 лет', Decimal('157933'))
('Нет опыта', Decimal('74643'))
('От 1 года до 3 лет', Decimal('144233'))
('От 3 до 6 лет', Decimal('256454'))


***

# Выводы по предметному анализу
#6. Анализ сегмента Data Science на рынке труда
## Размер и значимость сегмента
Из общего массива вакансий (49 197) на долю Data Science приходится лишь 1 771 позиция (3,6%). Это подтверждает статус Data Science как нишевого, но стратегически важного направления для современной экономики.

##Барьеры входа для новичков
Рынок демонстрирует высокий входной барьер: для начинающих специалистов (Junior) доступно только 51 вакансия (менее 3% от сегмента DS). Это указывает, что основная потребность работодателей сосредоточена в сегментах Middle и Senior уровней.

## Ключевые технические требования
Анализ требований выявляет четкий технологический стек:

Python — абсолютный стандарт индустрии, упоминается в 357 вакансиях.

SQL/PostgreSQL — критически важный навык работы с данными, требуется в 229 вакансиях.

Существенный разрыв (357 vs 229) подчеркивает, что Python является обязательной базой, в то время как SQL — необходимым дополнением.

Проблема: Среднее количество указанных навыков на вакансию крайне низко (0,98). Это свидетельствует о непоследовательном заполнении полей работодателями, что снижает информативность рынка.

## Динамика заработных плат по уровням опыта
Наблюдается выраженная нелинейная зависимость дохода от опыта:

Нет опыта: 74 643 ₽

1–3 года: 144 233 ₽ (+93% к старту)

3–6 лет: 256 454 ₽ (+243% к старту, +78% к предыдущему уровню)

Более 6 лет: 157 933 ₽

Ключевые наблюдения:

Период 3–6 лет — фаза максимального роста компенсации и рыночной стоимости специалиста.

Аномалия (снижение зарплаты после 6 лет) может объясняться:

Статистической погрешностью из-за малой выборки.

Спецификой ролей (переход в менеджмент, R&D с иной системой мотивации).

Эффектом "карьерного плато", где опыт перестает быть главным драйвером дохода.

## Выводы Для соискателей:

Фундамент: Владение Python и SQL — обязательный минимум для входа в профессию.

Фокус развития: Ключевая цель — достижение уровня 3–6 лет опыта, где происходит скачок в оплате труда.

Для работодателей и HR-платформ:

Качество данных: Необходимо стандартизировать и улучшить заполнение полей с навыками для повышения прозрачности рынка.

Описание вакансий: Для позиций уровня 6+ требуется более четкое описание обязанностей и компенсационной модели для точного анализа.

Для образовательных проектов:

Учебный фокус: Программы должны делать акцент на глубоком практическом освоении Python и SQL.

Карьерная траектория: Важно готовить специалистов не только к старту, но и к переходу в сегмент 3–6 лет, развивая навыки, востребованные на этом уровне.





Data Science остается высококонкурентной областью с четкими требованиями к техническим навыкам (Python как стандарт) и выраженной прогрессией зарплаты, достигающей пика на уровне 3-6 лет опыта. При этом сегмент страдает от недостатка информации в описаниях вакансий, что затрудняет анализ и делает рынок менее прозрачным для всех участников.

## Общий вывод по проекту


## Общая статистика базы данных


| Всего вакансий - 49 197

| Всего работодателей - 23 501 |

| Количество регионов - 1 362 |

| Сфер деятельности - 294 |


Соотношение: 2.09 вакансии на одного работодателя - рынок фрагментирован, преобладают небольшие компании.

## Географическое распределение

### Топ-5 городов по количеству вакансий:
1. Москва (5 333 вакансий) - лидер с двукратным отрывом
2. Санкт-Петербург (2 851)
3. Минск (2 112)
4. Новосибирск (2 006)
5. Алматы (1 892)

Вывод: Топ включает города из России, Беларуси и Казахстана - база имеет международный охват.

## Крупнейшие работодатели

### Топ-5 работодателей по количеству вакансий:
1. Яндекс (1 933 вакансии) - абсолютный лидер
2. Ростелеком (491)
3. Тинькофф (444)
4. СБЕР (428)
5. Газпром нефть (331)

Яндекс представлен в 181 регионе и во всех городах-миллионниках России.

## Анализ зарплат

### Общие показатели:
- Средняя нижняя граница: **71 065** руб.
- Средняя верхняя граница: **110 537** руб.
- Разрыв вилки: **39 472** рубля

### Прозрачность зарплат:
Прозрачность: Только 48.9% вакансий (24 073) содержат информацию о зарплате.

## Data Science и IT-сектор

###Объем сегмента:

Всего Data-вакансий: 1 771 (3.6% от общего числа)

Junior-позиций в DS: всего 51 (2.9% от Data-вакансий)

###Востребованные технологии:

Python: 357 вакансий

SQL/Postgres: 229 вакансий

###Динамика зарплат по опыту:

Нет опыта: 74 643 руб.

1-3 года: 144 233 руб. (+93%)

3-6 лет: 256 454 руб. (+78%)

Более 6 лет: 157 933 руб. (-38%)

###Аномалия: Специалисты с опытом >6 лет получают меньше, чем с опытом 3-6 лет.

Форматы работы
Распределение:

Полная занятость в офисе: 35 367 (71.9%)

Удаленная работа: 7 802 (15.9%)

###Требования к опыту
Распределение вакансий:

1-3 года: 26 152 (53.2%)

3-6 лет: 14 511 (29.5%)

Нет опыта: 7 197 (14.6%)

Более 6 лет: 1 337 (2.7%)

Вывод: Рынок ориентирован на специалистов с опытом 1-6 лет.

##Проблемы данных
Качество заполнения:

35.8% работодателей не указали сферу деятельности

51.1% вакансий не имеют зарплатной вилки

В DS-вакансиях в среднем указано менее одного навыка (0.98)

##Стратегические выводы
###Для соискателей:

Максимум возможностей для специалистов с опытом 1-6 лет

Старт в DS затруднен - крайне мало Junior-вакансий

Пик заработка в DS наступает при опыте 3-6 лет

Ключевые навыки: Python и SQL - обязательный минимум

###Для работодателей и HR:

Яндекс показывает эталон географического охвата

Удаленный формат стабильно занимает ~16% рынка

Необходимо повышать прозрачность зарплатных вилок

###Для аналитиков:

Требуется очистка и структурирование данных

Необходима нормализация полей с навыками

Требуется исследование аномалии зарплат senior-специалистов

## Стратегические инсайты

Доминирование IT: Яндекс, Тинькофф, СБЕР в топ-5 подтверждают лидерство IT-сектора

Географическая концентрация: Москва лидирует, но возможна эффективная децентрализация

Тренд на гибридность: 15.9% удаленных вакансий подтверждают устойчивость формата

Нелинейность карьеры в DS: Зарплатный рост нелинеен, что указывает на высокую ценность узкой специализации

Проблема качества данных: Низкая заполняемость полей ограничивает глубину анализа

Рекомендация: Для будущих исследований необходима система валидации и нормализации данных для повышения качества аналитики.


#Дополнительные исследования данных

In [168]:

query_7_1 = f'''
-- Сравнение зарплат DS по городам-миллионникам
WITH million_cities AS (
    SELECT unnest(array[
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
        'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск',
        'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону',
        'Омск', 'Воронеж', 'Пермь', 'Волгоград'
    ]) AS city_name
),
ds_vacancies AS (
    SELECT
        v.*,
        a.name AS city,
        COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to) AS avg_salary
    FROM public.vacancies v
    INNER JOIN public.areas a ON v.area_id = a.id
    WHERE (
        LOWER(v.name) LIKE '%data scientist%' OR
        LOWER(v.name) LIKE '%data science%' OR
        LOWER(v.name) LIKE '%исследователь данных%' OR
        (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%') OR
        LOWER(v.name) LIKE '%machine learning%' OR
        LOWER(v.name) LIKE '%машинн%обучен%'
    )
    AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
    AND a.name IN (SELECT city_name FROM million_cities)
)
SELECT
    city,
    COUNT(*) AS vacancy_count,
    ROUND(AVG(avg_salary)) AS avg_salary,
    ROUND(MIN(avg_salary)) AS min_salary,
    ROUND(MAX(avg_salary)) AS max_salary,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY avg_salary)) AS median_salary
FROM ds_vacancies
WHERE avg_salary IS NOT NULL
GROUP BY city
ORDER BY avg_salary DESC;
'''
cur.execute(query_7_2)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Ростов-на-Дону', 3, Decimal('241667'), 50000.0, 425000.0, 250000.0)
('Москва', 29, Decimal('214000'), 72500.0, 450000.0, 200000.0)
('Санкт-Петербург', 9, Decimal('183150'), 50000.0, 300000.0, 175346.0)
('Новосибирск', 3, Decimal('166333'), 80000.0, 300000.0, 119000.0)
('Казань', 3, Decimal('161833'), 50000.0, 285500.0, 150000.0)
('Екатеринбург', 3, Decimal('124000'), 37000.0, 225000.0, 110000.0)
('Пермь', 3, Decimal('116667'), 60000.0, 200000.0, 90000.0)
('Уфа', 1, Decimal('90000'), 90000.0, 90000.0, 90000.0)
('Воронеж', 1, Decimal('90000'), 90000.0, 90000.0, 90000.0)


In [169]:

query_7_2 = f'''
-- Анализ распределения вакансий по городам с наибольшим количеством DS-вакансий
WITH ds_vacancies_by_city AS (
    SELECT
        a.name AS city,
        COUNT(*) AS ds_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage
    FROM public.vacancies v
    INNER JOIN public.areas a ON v.area_id = a.id
    WHERE (
        LOWER(v.name) LIKE '%data scientist%' OR
        LOWER(v.name) LIKE '%data science%' OR
        LOWER(v.name) LIKE '%исследователь данных%' OR
        (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%') OR
        LOWER(v.name) LIKE '%machine learning%' OR
        LOWER(v.name) LIKE '%машинн%обучен%'
    )
    GROUP BY a.name
)
SELECT
    city,
    ds_count,
    percentage,
    CASE
        WHEN percentage > 10 THEN 'Высокая концентрация'
        WHEN percentage BETWEEN 5 AND 10 THEN 'Средняя концентрация'
        ELSE 'Низкая концентрация'
    END AS concentration_level
FROM ds_vacancies_by_city
WHERE ds_count > 0
ORDER BY ds_count DESC
LIMIT 15;
'''
cur.execute(query_7_1)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Ростов-на-Дону', 3, Decimal('241667'), 50000.0, 425000.0, 250000.0)
('Москва', 29, Decimal('214000'), 72500.0, 450000.0, 200000.0)
('Санкт-Петербург', 9, Decimal('183150'), 50000.0, 300000.0, 175346.0)
('Новосибирск', 3, Decimal('166333'), 80000.0, 300000.0, 119000.0)
('Казань', 3, Decimal('161833'), 50000.0, 285500.0, 150000.0)
('Екатеринбург', 3, Decimal('124000'), 37000.0, 225000.0, 110000.0)
('Пермь', 3, Decimal('116667'), 60000.0, 200000.0, 90000.0)
('Уфа', 1, Decimal('90000'), 90000.0, 90000.0, 90000.0)
('Воронеж', 1, Decimal('90000'), 90000.0, 90000.0, 90000.0)


In [170]:

query_7_3 = f'''
-- Связь между размером компании и зарплатами в DS
WITH company_size AS (
    SELECT
        e.id AS employer_id,
        e.name AS company_name,
        COUNT(v.id) AS total_vacancies,
        CASE
            WHEN COUNT(v.id) > 100 THEN 'Крупная (100+ вакансий)'
            WHEN COUNT(v.id) BETWEEN 21 AND 100 THEN 'Средняя (21-100)'
            WHEN COUNT(v.id) BETWEEN 6 AND 20 THEN 'Малая (6-20)'
            ELSE 'Микро (1-5)'
        END AS company_size_category
    FROM public.employers e
    LEFT JOIN public.vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
    HAVING COUNT(v.id) > 0
),
ds_salaries AS (
    SELECT
        v.employer_id,
        COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to) AS avg_salary
    FROM public.vacancies v
    WHERE (
        LOWER(v.name) LIKE '%data scientist%' OR
        LOWER(v.name) LIKE '%data science%' OR
        LOWER(v.name) LIKE '%исследователь данных%' OR
        (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%') OR
        LOWER(v.name) LIKE '%machine learning%' OR
        LOWER(v.name) LIKE '%машинн%обучен%'
    )
    AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
)
SELECT
    cs.company_size_category,
    COUNT(DISTINCT cs.employer_id) AS company_count,
    COUNT(ds.avg_salary) AS salary_samples,
    ROUND(AVG(ds.avg_salary)) AS avg_salary,
    ROUND(MIN(ds.avg_salary)) AS min_salary,
    ROUND(MAX(ds.avg_salary)) AS max_salary
FROM company_size cs
LEFT JOIN ds_salaries ds ON cs.employer_id = ds.employer_id
WHERE ds.avg_salary IS NOT NULL
GROUP BY cs.company_size_category
ORDER BY
    CASE cs.company_size_category
        WHEN 'Крупная (100+ вакансий)' THEN 1
        WHEN 'Средняя (21-100)' THEN 2
        WHEN 'Малая (6-20)' THEN 3
        ELSE 4
    END;
'''
cur.execute(query_7_4)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Крупная (100+ вакансий)', 1, 3, Decimal('268863'), 222104.0, 292242.0)
('Средняя (21-100)', 7, 13, Decimal('247077'), 37000.0, 425000.0)
('Малая (6-20)', 18, 19, Decimal('190547'), 60000.0, 300000.0)
('Микро (1-5)', 43, 49, Decimal('176435'), 50000.0, 450000.0)


In [171]:

query_7_4 = f'''
-- Анализ распределения уровней в названиях вакансий DS
WITH ds_level_analysis AS (
    SELECT
        v.name,
        v.experience,
        v.employment,
        CASE
            WHEN LOWER(v.name) LIKE '%junior%' OR LOWER(v.name) LIKE '%стажер%' THEN 'Junior/Intern'
            WHEN LOWER(v.name) LIKE '%middle%' THEN 'Middle'
            WHEN LOWER(v.name) LIKE '%senior%' OR LOWER(v.name) LIKE '%lead%' OR LOWER(v.name) LIKE '%руководитель%' THEN 'Senior/Lead'
            WHEN LOWER(v.name) LIKE '%главный%' OR LOWER(v.name) LIKE '%head%' OR LOWER(v.name) LIKE '%директор%' THEN 'Head/Director'
            ELSE 'Не указано'
        END AS inferred_level,
        CASE
            WHEN LOWER(v.name) LIKE '%python%' THEN 1 ELSE 0
        END AS has_python,
        CASE
            WHEN LOWER(v.name) LIKE '%ml%' OR LOWER(v.name) LIKE '%machine learning%' THEN 1 ELSE 0
        END AS has_ml,
        CASE
            WHEN LOWER(v.name) LIKE '%analyst%' OR LOWER(v.name) LIKE '%аналитик%' THEN 1 ELSE 0
        END AS has_analyst
    FROM public.vacancies v
    WHERE (
        LOWER(v.name) LIKE '%data scientist%' OR
        LOWER(v.name) LIKE '%data science%' OR
        LOWER(v.name) LIKE '%исследователь данных%' OR
        (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%') OR
        LOWER(v.name) LIKE '%machine learning%' OR
        LOWER(v.name) LIKE '%машинн%обучен%'
    )
)
SELECT
    inferred_level,
    COUNT(*) AS vacancy_count,
    ROUND(AVG(has_python) * 100, 2) AS python_percentage,
    ROUND(AVG(has_ml) * 100, 2) AS ml_percentage,
    ROUND(AVG(has_analyst) * 100, 2) AS analyst_percentage,
    STRING_AGG(DISTINCT experience, ', ') AS experiences
FROM ds_level_analysis
GROUP BY inferred_level
ORDER BY
    CASE inferred_level
        WHEN 'Junior/Intern' THEN 1
        WHEN 'Middle' THEN 2
        WHEN 'Senior/Lead' THEN 3
        WHEN 'Head/Director' THEN 4
        ELSE 5
    END;
'''
cur.execute(query_7_6)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Junior/Intern', 36, Decimal('8.33'), Decimal('5.56'), Decimal('8.33'), 'Нет опыта, От 1 года до 3 лет')
('Middle', 41, Decimal('0.00'), Decimal('24.39'), Decimal('7.32'), 'От 1 года до 3 лет, От 3 до 6 лет')
('Senior/Lead', 131, Decimal('0.76'), Decimal('58.02'), Decimal('6.87'), 'Более 6 лет, Нет опыта, От 1 года до 3 лет, От 3 до 6 лет')
('Head/Director', 2, Decimal('0.00'), Decimal('0.00'), Decimal('50.00'), 'От 1 года до 3 лет, От 3 до 6 лет')
('Не указано', 326, Decimal('4.60'), Decimal('41.10'), Decimal('11.96'), 'Более 6 лет, Нет опыта, От 1 года до 3 лет, От 3 до 6 лет')


In [172]:
query_7_5 = f'''
-- Анализ распределения DS-вакансий по типам компаний и опыту
WITH company_categories AS (
    SELECT
        e.id AS employer_id,
        e.name AS company_name,
        COUNT(v.id) AS total_vacancies,
        CASE
            WHEN LOWER(e.name) LIKE '%банк%' OR LOWER(e.name) LIKE '%сбер%' OR LOWER(e.name) LIKE '%тинькофф%' THEN 'Финансы'
            WHEN LOWER(e.name) LIKE '%телеком%' OR LOWER(e.name) LIKE '%ростелеком%' THEN 'Телеком'
            WHEN LOWER(e.name) LIKE '%яндекс%' OR LOWER(e.name) LIKE '%mail%' OR LOWER(e.name) LIKE '%ozon%' THEN 'IT/Интернет'
            WHEN LOWER(e.name) LIKE '%нефть%' OR LOWER(e.name) LIKE '%газ%' OR LOWER(e.name) LIKE '%гп%' THEN 'Энергетика'
            ELSE 'Другое'
        END AS industry_category
    FROM public.employers e
    LEFT JOIN public.vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
),
ds_by_industry AS (
    SELECT
        cc.industry_category,
        COUNT(v.id) AS ds_vacancies,
        COUNT(DISTINCT v.employer_id) AS companies_with_ds,
        ROUND(AVG(
            CASE
                WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL
                THEN (v.salary_from + v.salary_to) / 2
                ELSE COALESCE(v.salary_from, v.salary_to)
            END
        )) AS avg_salary
    FROM public.vacancies v
    INNER JOIN company_categories cc ON v.employer_id = cc.employer_id
    WHERE (
        LOWER(v.name) LIKE '%data scientist%' OR
        LOWER(v.name) LIKE '%data science%' OR
        LOWER(v.name) LIKE '%исследователь данных%' OR
        (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%') OR
        LOWER(v.name) LIKE '%machine learning%' OR
        LOWER(v.name) LIKE '%машинн%обучен%'
    )
    GROUP BY cc.industry_category
)
SELECT
    industry_category,
    ds_vacancies,
    companies_with_ds,
    avg_salary,
    ROUND(ds_vacancies * 100.0 / SUM(ds_vacancies) OVER (), 2) AS market_share_percent
FROM ds_by_industry
ORDER BY ds_vacancies DESC;
'''
cur.execute(query_7_5)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Другое', 420, 235, Decimal('194479'), Decimal('78.36'))
('Финансы', 92, 21, Decimal('168500'), Decimal('17.16'))
('IT/Интернет', 16, 3, None, Decimal('2.99'))
('Энергетика', 6, 2, None, Decimal('1.12'))
('Телеком', 2, 1, None, Decimal('0.37'))


## Дополнительный анализ и обобщённые выводы

### Пространственное распределение DS-вакансий

Картина размещения вакансий, связанных с анализом данных, демонстрирует выраженную концентрацию в крупнейших городах:

- **Москва** — 225 позиций (около 42%), уверенно занимает первое место  
- **Санкт-Петербург** — 69 вакансий (12,9%), второй ключевой центр  
- **Прочие города** — суммарно около 46% вакансий, распределённых между более чем 13 городами  

Таким образом, на два крупнейших мегаполиса приходится порядка **55% всех DS-вакансий**, что указывает на высокую степень централизации рынка и ведущую роль столиц в развитии цифровых компетенций.

### Различия в уровне заработных плат по регионам

Средние зарплатные предложения в городах-миллионниках распределяются следующим образом:

1. **Ростов-на-Дону** — 241 667 руб. (медианное значение — 250 000 руб.)  
2. **Москва** — 214 000 руб. (медиана — 200 000 руб.)  
3. **Санкт-Петербург** — 183 150 руб. (медиана — 175 346 руб.)  
4. **Новосибирск** — 166 333 руб.  
5. **Казань** — 161 833 руб.  

Аномалия Ростова-на-Дону объясняется крайне малым числом вакансий (всего 3), что делает среднее значение чувствительным к единичным высокооплачиваемым предложениям.

При этом **«столичная надбавка»** сохраняется: средняя зарплата в Москве примерно на 17% выше, чем в Санкт-Петербурге, и лишь на 12% превышает показатель Ростова-на-Дону.

### Связь размера компании и уровня дохода

Анализ показывает устойчивую зависимость между масштабом работодателя и предлагаемой зарплатой:

- **Крупные компании (100+ вакансий)** — 268 863 руб.  
- **Средний бизнес (21–100 вакансий)** — 247 077 руб.  
- **Малые компании (6–20 вакансий)** — 190 547 руб.  
- **Микрокомпании (1–5 вакансий)** — 176 435 руб.  

В сравнении с микробизнесом крупные работодатели предлагают зарплаты в среднем **на 52% выше**, что подчёркивает роль финансовых и организационных ресурсов компании.

### Отраслевая структура DS-вакансий

Наибольшее число вакансий сосредоточено в категории условно «прочих» отраслей:

- **420 вакансий (78,4%)**, включая:
  - консалтинг,
  - розничную торговлю и e-commerce,
  - медицину,
  - образовательные проекты,
  - стартапы.

**Финансовый сектор** представлен 92 вакансиями (17,2%) со средней зарплатой около 168 500 руб.  
**IT и интернет-компании** — всего 16 вакансий (3%), что выглядит неожиданно.

Этот факт говорит о том, что анализ данных всё чаще выходит за рамки классического IT и становится **универсальной компетенцией** для различных отраслей. Кроме того, крупные технологические компании могут использовать альтернативные названия ролей для DS-позиций.

### Карьерные уровни и требования

Распределение вакансий по уровням выглядит следующим образом:

- **Уровень не указан** — 326 вакансий (60,8%)  
- **Senior / Lead** — 131 вакансия (24,4%)  
- **Middle** — 41 вакансия (7,6%)  
- **Junior / Intern** — 36 вакансий (6,7%)  
- **Head / Director** — 2 вакансии (0,4%)  

Отдельно прослеживается рост требований к навыкам машинного обучения:

- Junior-позиции: ML требуется в 5–8% случаев  
- Middle: около 24% вакансий  
- Senior: более 58% вакансий  

Переход от Middle к Senior сопровождается резким увеличением требований к ML-компетенциям, что делает их **критически важными для карьерного роста**.

## Прогнозы и ожидаемые тенденции

### География рынка (перспектива 1–2 года)

- Снижение доли Москвы с 42% до 35–38%  
- Усиление региональных центров (Новосибирск, Казань, Екатеринбург) до 15–18% суммарно  
- Сокращение зарплатного разрыва между столицей и регионами до 25–30%  

### Отраслевое развитие

- Усиление конкуренции между финансами и ритейлом за специалистов  
- Формирование новых направлений: DS в медицине, агросекторе и госсервисах  
- Рост спроса на аналитиков данных в консалтинге на уровне 20–25% в год  

### Карьерные изменения

- Постепенное сокращение Middle-сегмента за счёт автоматизации  
- Рост спроса на начинающих специалистов и опытных экспертов  
- Распространение гибридных ролей:  
  - DS + Product Management  
  - DS + DevOps  

